In [1]:
# Prior to this, peptides have already been filtered by library design in Step 0
# This notebook will take AS-MS data that has been filtered by library design and rigorously remove the sequence isomers

In [1]:
import os, difflib
import pandas as pd
import numpy as np

In [2]:
# file_list = [],
# for file in os.listdir(os.getcwd()):
#     if file.endswith('.csv'):
#         file_list.append(file)

In [3]:
file = 'Example Raw Data.csv'
df = pd.read_csv(file)

In [4]:
df

,Notebook,Scan,Peptide,ALC (%),m/z,z,Obs Parent Mass,RT,Mass,ppm
0,284-126,3203,NNGGGLGSGQASK,89,573.2864,2,1144.5582,22.55,1144.5586,-0.3
1,268-020,5770,TGSAADVADYSAK,96,627.7993,2,1253.5840,27.06,1253.5889,-3.8
2,268-186,318,GSASGLSFKGSHK,99,421.2267,3,1260.6582,9.91,1260.6575,0.7
3,256-092,10187,GMASDMADYAAGK,99,643.7775,2,1285.5404,43.45,1285.5432,-2.2
4,268-175,4195,GLASGLSFKGSHK,99,429.9094,3,1286.7063,25.99,1286.7095,-2.5
...,...,...,...,...,...,...,...,...,...,...
4262,284-079,12732,WYTHMMFPWMWFK,95,630.6179,3,1888.8318,74.98,1888.8403,-4.4
4263,261-179,26317,LYYYMTPWWPYWK,95,948.4512,2,1894.8878,87.87,1894.8906,-1.5
4264,228-002,38310,YLYYMTPWPWYWK,98,956.4479,2,1910.8812,96.49,1910.8855,-2.2
4265,271-146,5783,WRNKFWLQWHWYK,97,495.0081,4,1976.0032,48.36,1976.0110,-3.9


In [5]:
# Filter out based on ALC, ppm, and Uniqueness create a new dataframe
ALC_Cutoff = 85 # inclusive of this number and higher
ppm_error_cutoff = 10 # exclusive of this number, i.e., if 10, then 9.9 ppm is kept

# ALC, which is sequencing confidence, This was previously filtered, and is filtered again here
ALC_list = df['ALC (%)'].to_list()
Good_ALC_list = []
for i,ALC in enumerate(ALC_list):
    if ALC > ALC_Cutoff-1: Good_ALC_list.append(i)
df_ALC = df.iloc[Good_ALC_list,:]

# ppm error
ppm_list = df_ALC['ppm'].to_list()
Good_ppm_list = []
for i,ppm in enumerate(ppm_list):
    if ppm > -ppm_error_cutoff and ppm < ppm_error_cutoff: Good_ppm_list.append(i)
df = df_ALC.iloc[Good_ppm_list,:]

# # Use this if it is needed
# # Remove nonunique peptides
# Unique_list = df_ppm['Uniqueness'].to_list()
# Good_unique_list = []
# for i,uniq in enumerate(Unique_list):
#     if uniq == 'UNIQUE': Good_unique_list.append(i)
# df = df_ppm.iloc[Good_unique_list,:] # overwrites earlier dataframe

# Calculate and sort by observed parent mass, OPM
m_z = df['m/z'].to_list()
zs = df['z'].to_list()
OPM = []
for i,mz in enumerate(m_z):
    OPM.append(mz*zs[i]-zs[i]*1.0073)
df['Obs Parent Mass'] = OPM
df = df.sort_values(by=['Obs Parent Mass']) # technically unnecessary, but useful to allow manual inspection

df.reset_index(drop=True,inplace=True) # required such that the indices of lists and dataframes match

# Clean up peptides for comparison, remove KU if it exists, remove ' 's that sometime exist
peptides_start = df['Peptide'].tolist()
peptides = [pep.replace('KU', 'K') for pep in peptides_start]
betterpeptides = [string for string in peptides if string != '']
df['Peptide'] = betterpeptides
print(f'ALC, ppm, and Unique filtering completed, removed {len(ALC_list)-len(peptides_start)} peptides')

ALC, ppm, and Unique filtering completed, removed 66 peptides


In [6]:
df

,Notebook,Scan,Peptide,ALC (%),m/z,z,Obs Parent Mass,RT,Mass,ppm
0,284-126,3203,NNGGGLGSGQASK,89,573.2864,2,1144.5582,22.55,1144.5586,-0.3
1,268-020,5770,TGSAADVADYSAK,96,627.7993,2,1253.5840,27.06,1253.5889,-3.8
2,268-186,318,GSASGLSFKGSHK,99,421.2267,3,1260.6582,9.91,1260.6575,0.7
3,256-092,10187,GMASDMADYAAGK,99,643.7775,2,1285.5404,43.45,1285.5432,-2.2
4,268-175,4195,GLASGLSFKGSHK,99,429.9094,3,1286.7063,25.99,1286.7095,-2.5
...,...,...,...,...,...,...,...,...,...,...
4196,284-079,12732,WYTHMMFPWMWFK,95,630.6179,3,1888.8318,74.98,1888.8403,-4.4
4197,261-179,26317,LYYYMTPWWPYWK,95,948.4512,2,1894.8878,87.87,1894.8906,-1.5
4198,228-002,38310,YLYYMTPWPWYWK,98,956.4479,2,1910.8812,96.49,1910.8855,-2.2
4199,271-146,5783,WRNKFWLQWHWYK,97,495.0081,4,1976.0032,48.36,1976.0110,-3.9


In [7]:
# Step 1, remove peptide duplicates
df = df.drop_duplicates(subset=['Peptide'])
df.reset_index(drop=True,inplace=True)
print(f'Step 1 completed, removed {len(peptides_start)-len(df.index)} duplicates')

Step 1 completed, removed 0 duplicates


In [8]:
df

,Notebook,Scan,Peptide,ALC (%),m/z,z,Obs Parent Mass,RT,Mass,ppm
0,284-126,3203,NNGGGLGSGQASK,89,573.2864,2,1144.5582,22.55,1144.5586,-0.3
1,268-020,5770,TGSAADVADYSAK,96,627.7993,2,1253.5840,27.06,1253.5889,-3.8
2,268-186,318,GSASGLSFKGSHK,99,421.2267,3,1260.6582,9.91,1260.6575,0.7
3,256-092,10187,GMASDMADYAAGK,99,643.7775,2,1285.5404,43.45,1285.5432,-2.2
4,268-175,4195,GLASGLSFKGSHK,99,429.9094,3,1286.7063,25.99,1286.7095,-2.5
...,...,...,...,...,...,...,...,...,...,...
4196,284-079,12732,WYTHMMFPWMWFK,95,630.6179,3,1888.8318,74.98,1888.8403,-4.4
4197,261-179,26317,LYYYMTPWWPYWK,95,948.4512,2,1894.8878,87.87,1894.8906,-1.5
4198,228-002,38310,YLYYMTPWPWYWK,98,956.4479,2,1910.8812,96.49,1910.8855,-2.2
4199,271-146,5783,WRNKFWLQWHWYK,97,495.0081,4,1976.0032,48.36,1976.0110,-3.9


In [9]:
# Step 2: Remove sequence isomers with the same precursor mass -OR- with a different specific precursor mass
# There are multiple situations where the removal of precursors of different masses is justified including:
    # 1) incorrect monoisotopic precursor selection (abs delta of 1)
    # 2) oxidations (abs delta of 16, 32)
    # 3) and sodium adduct (abs delta of 22)
# 0.69 similarity was emperically determined and is very lenient in the comparison (multiple mismatches / shifts allowed)
# the mass differences are the tight requirement here; and thus, and thus, ALC cutoffs are not considered
# RT differences are not considered in case the data was acquired using different gradients, though they were used to set the similarity requirement
# The highest ALC peptide will be retained, with ppm as tie-breaker

# Extract lists from dataframe for enumeration
peptides = df['Peptide'].tolist()
OPM = df['Obs Parent Mass'].tolist()
ALC = df['ALC (%)'].tolist()
ppm = df['ppm'].tolist()

# Make a list of all the peptides to remove, Step 2:
similarity_cutoff = 0.69      # more than this
Mass_diff_list = [0,1,16,32,22] # and a mass difference within this list

to_drop = []
for i in range(len(peptides)):
    if len(peptides) > 500 and i % 100 == 0: print(f"Step 2: {i} of {len(peptides)}")
    for k in range(len(peptides)):
        sim = difflib.SequenceMatcher(a=peptides[k], b=peptides[i])
        simz = sim.ratio()
        difference = np.abs(float(OPM[i]) - float(OPM[k]))
        if i != k and simz > similarity_cutoff:
            # print(f"Trouble {peptides[k]} {ALC[k]} {round(float(OPM[k]),3)} and {peptides[i]} {ALC[i]} {round(float(OPM[i]),3)}, similarity {round(float(simz),3)} difference {round(float(difference),3)}")
            if round(float(difference)) in Mass_diff_list:
                if ALC[k] > ALC[i]:
                    print(f"1: I've marked {peptides[i]} {ALC[i]}, index is {i}, similarity {round(float(simz),3)}, Mass difference {round(float(difference),3)}")
                    to_drop.append(i)
                elif ALC[k] < ALC[i]:
                    print(f"2: I've marked {peptides[k]} {ALC[k]}, index is {k}, similarity {round(float(simz),3)}, Mass difference {round(float(difference),3)}")
                    to_drop.append(k)
                elif ALC[k] == ALC[i]:    
                    if abs(float(ppm[k])) < abs(float(ppm[i])):
                        print(f"3: I've marked {peptides[k]} {ALC[k]}, index is {k}, similarity {round(float(simz),3)}, Mass difference {round(float(difference),3)}")
                        to_drop.append(k)
                    elif abs(float(ppm[i])) < abs(float(ppm[k])):
                        print(f"4: I've marked {peptides[i]} {ALC[i]}, index is {i}, similarity {round(float(simz),3)}, Mass difference {round(float(difference),3)}")
                        to_drop.append(i)
                    else:
                        print(f"5: I've marked {peptides[k]} {ALC[k]}, index is {k}, similarity {round(float(simz),3)}, Mass difference {round(float(difference),3)}")
                        to_drop.append(k)

Step 2: 0 of 4201
2: I've marked DLADYAAVDGGFK 91, index is 19, similarity 0.692, Mass difference 32.004
1: I've marked LTTGDAPDYADGK 88, index is 10, similarity 0.692, Mass difference 21.954
1: I've marked PDVPDYAVASGDK 90, index is 14, similarity 0.692, Mass difference 32.064
3: I've marked APGDLPDYAVTNK 95, index is 27, similarity 0.692, Mass difference 22.094
2: I've marked NSFEGDAPDYAGK 90, index is 43, similarity 0.692, Mass difference 32.007
1: I've marked DLADYAAVDGGFK 91, index is 19, similarity 0.692, Mass difference 32.004
1: I've marked DLADYAAVDGGFK 91, index is 19, similarity 0.692, Mass difference 15.996
2: I've marked LTTGDAPDYADGK 88, index is 10, similarity 0.692, Mass difference 21.954
4: I've marked SVDTGDMADYAGK 95, index is 20, similarity 0.692, Mass difference 15.977
1: I've marked PGQNLDVGDYAAK 96, index is 21, similarity 0.692, Mass difference 21.981
1: I've marked QGSADLPDYAANK 97, index is 22, similarity 0.692, Mass difference 31.992
4: I've marked GGVLRDVSDY

2: I've marked HKYGADVVDYAAK 92, index is 245, similarity 0.692, Mass difference 22.105
3: I've marked DPGTYDLVDYAGK 99, index is 140, similarity 0.692, Mass difference 1.009
3: I've marked VGDPGDLHDYAEK 99, index is 150, similarity 0.692, Mass difference 0.981
2: I've marked VNGTDDLTDYSAK 96, index is 100, similarity 0.692, Mass difference 16.087
4: I've marked VGDPGDLHDYAEK 99, index is 150, similarity 0.692, Mass difference 0.981
4: I've marked VGDPGDLHDYAEK 99, index is 150, similarity 0.692, Mass difference 16.013
2: I've marked VGEHDDLEDYAGK 98, index is 317, similarity 0.692, Mass difference 31.99
2: I've marked LVGGLFDLHDYAK 93, index is 322, similarity 0.769, Mass difference 32.111
2: I've marked SQADLPDYSATVK 96, index is 90, similarity 0.692, Mass difference 21.931
1: I've marked ASLSYDVQDYAGK 97, index is 154, similarity 0.692, Mass difference 22.013
3: I've marked TSMEDVADYADGK 99, index is 105, similarity 0.923, Mass difference 15.994
1: I've marked QSPVDLADYSAQK 92, inde

2: I've marked MLDAPDYAQDSAK 87, index is 187, similarity 0.846, Mass difference 15.994
1: I've marked MLDAPDYANTDAK 93, index is 266, similarity 0.692, Mass difference 15.99
1: I've marked YPWALDAADYAGK 95, index is 268, similarity 0.769, Mass difference 32.022
1: I've marked YPWALDAADYAGK 95, index is 268, similarity 0.692, Mass difference 22.081
1: I've marked GPVQGDVMPYAYK 88, index is 269, similarity 0.846, Mass difference 1.001
4: I've marked WGSLTDLTDYAAK 99, index is 270, similarity 0.692, Mass difference 22.056
3: I've marked SSDVDAPDYAELK 99, index is 129, similarity 0.692, Mass difference 32.003
4: I've marked EFDLADYAATTPK 99, index is 275, similarity 0.692, Mass difference 0.994
2: I've marked EGVFDLADYAMDK 85, index is 521, similarity 0.692, Mass difference 31.969
4: I've marked VSAGDMEDYAEQK 93, index is 276, similarity 0.846, Mass difference 15.991
3: I've marked PYLANDVADYAAK 99, index is 133, similarity 0.692, Mass difference 31.952
3: I've marked EFDLADYAATTPK 99, in

3: I've marked LVLFDATDYADSK 99, index is 382, similarity 0.692, Mass difference 1.022
3: I've marked QSLTDWADYAASK 99, index is 370, similarity 0.692, Mass difference 1.023
3: I've marked LVLFDATDYADSK 99, index is 382, similarity 0.692, Mass difference 1.019
4: I've marked YGDDVQDYAQVGK 99, index is 380, similarity 0.692, Mass difference 0.997
1: I've marked DGVPDYAADLYMK 85, index is 381, similarity 0.692, Mass difference 1.036
4: I've marked LVLFDATDYADSK 99, index is 382, similarity 0.692, Mass difference 1.022
4: I've marked LVLFDATDYADSK 99, index is 382, similarity 0.692, Mass difference 1.019
2: I've marked EGVFDLADYAMDK 85, index is 521, similarity 0.692, Mass difference 15.938
3: I've marked EPVQNDVADYSAK 99, index is 235, similarity 0.692, Mass difference 22.056
2: I've marked VTPGFDLEDYAAK 98, index is 194, similarity 0.692, Mass difference 32.039
3: I've marked LGSDLEDYAHTMK 99, index is 577, similarity 0.692, Mass difference 21.95
4: I've marked VPLGTHDLEDYAK 99, index i

2: I've marked LAANMVDYADARK 86, index is 358, similarity 0.692, Mass difference 15.908
2: I've marked RMDDYADAVGNVK 89, index is 356, similarity 0.846, Mass difference 15.991
1: I've marked DMRDYADAVGNVK 91, index is 479, similarity 0.692, Mass difference 31.954
2: I've marked MGADLHDYAAFMK 98, index is 843, similarity 0.769, Mass difference 31.94
4: I've marked LLDFPDYAFGLAK 96, index is 484, similarity 0.692, Mass difference 16.004
3: I've marked WLGLDDTSDYAAK 99, index is 363, similarity 0.769, Mass difference 15.973
2: I've marked MPFSDAPDYADNK 92, index is 647, similarity 0.692, Mass difference 15.927
3: I've marked AAEQDVFDYADDK 99, index is 649, similarity 0.692, Mass difference 15.925
3: I've marked AFGGEDVEDYAWK 99, index is 652, similarity 0.692, Mass difference 15.942
4: I've marked YDMDVADYASGHK 99, index is 492, similarity 0.692, Mass difference 22.057
1: I've marked MGLPTDHHDYAAK 93, index is 494, similarity 0.692, Mass difference 32.02
2: I've marked FGLPTDHHDYAAK 90, i

...

2: I've marked LDRHDYAHFQQLK 97, index is 3496, similarity 0.692, Mass difference 0.077
2: I've marked YFFMDLPDYANFK 95, index is 3660, similarity 0.692, Mass difference 15.938
1: I've marked LDRHDYAHFQQLK 97, index is 3496, similarity 0.692, Mass difference 0.077
1: I've marked HFTDMQDYSDREK 94, index is 3498, similarity 0.692, Mass difference 16.047
2: I've marked WTDLQDYSFDVRK 90, index is 3510, similarity 0.692, Mass difference 1.084
Step 2: 3500 of 4201
2: I've marked FKRWDKPDYAADK 95, index is 3069, similarity 0.692, Mass difference 32.028
1: I've marked FYMHDAFQDYAHK 90, index is 3506, similarity 0.692, Mass difference 1.003
3: I've marked DFDKEDYARVWTK 98, index is 3511, similarity 0.692, Mass difference 0.039
1: I've marked WTDLQDYSFDVRK 90, index is 3510, similarity 0.692, Mass difference 1.084
4: I've marked DFDKEDYARVWTK 98, index is 3511, similarity 0.692, Mass difference 0.039
1: I've marked WKLQYDQQDYASK 94, index is 3512, similarity 0.692, Mass difference 31.979
2: I've

2: I've marked RNFDRRDRDYAAK 88, index is 3621, similarity 0.692, Mass difference 0.001
1: I've marked RNFDRRDRDYAAK 88, index is 3621, similarity 0.692, Mass difference 0.001
2: I've marked YVYDMMDYAHNTK 91, index is 3613, similarity 0.692, Mass difference 0.999
2: I've marked TRLWDVHDYATFK 97, index is 3232, similarity 0.692, Mass difference 31.961
2: I've marked FDWHDYAEHVQSK 90, index is 3357, similarity 0.692, Mass difference 22.056
4: I've marked FLYDVHDYAFEHK 99, index is 3627, similarity 0.769, Mass difference 1.053
1: I've marked ERDLHDYAFYLNK 98, index is 3629, similarity 0.692, Mass difference 16.132
1: I've marked ERDLHDYAFYLNK 98, index is 3629, similarity 0.846, Mass difference 0.008
4: I've marked VWDLHDYAFYLNK 99, index is 3630, similarity 0.692, Mass difference 16.14
2: I've marked ERDLHDYAFYLNK 98, index is 3629, similarity 0.846, Mass difference 0.008
1: I've marked TLYSWDMEDYAMK 86, index is 3634, similarity 0.923, Mass difference 31.988
1: I've marked TLYSWDMEDYAMK

4: I've marked DWDSWDKQDYAHK 99, index is 3711, similarity 0.692, Mass difference 1.031
1: I've marked WRMGFDVHDYSHK 95, index is 3712, similarity 0.692, Mass difference 0.035
2: I've marked MYKDLYDYADKYK 96, index is 3884, similarity 0.692, Mass difference 22.03
4: I've marked YLFDLQDYAFDRK 99, index is 3718, similarity 0.692, Mass difference 32.019
3: I've marked WNDQKDYAWNVKK 96, index is 3736, similarity 0.692, Mass difference 0.95
1: I've marked WDDDEMDYAEMDK 91, index is 3720, similarity 0.692, Mass difference 1.103
2: I've marked WDFKDSYEDYAEK 89, index is 3739, similarity 0.692, Mass difference 1.14
1: I've marked FSEDQHDYSEFYK 92, index is 3721, similarity 0.692, Mass difference 16.079
3: I've marked FQYFDMHDYASQK 99, index is 3738, similarity 0.692, Mass difference 0.996
4: I've marked FNFYDMHDYAEVK 99, index is 3722, similarity 0.692, Mass difference 1.016
1: I've marked PEMYVYDREDYAK 90, index is 3723, similarity 0.923, Mass difference 15.995
1: I've marked VSQEYDYQDYAWK 93

2: I've marked YYDVADYAFWEHK 86, index is 3823, similarity 0.692, Mass difference 0.066
1: I've marked MNDYWDYASDHYK 89, index is 3828, similarity 0.692, Mass difference 0.066
2: I've marked MNDYWDYASDHYK 89, index is 3828, similarity 0.692, Mass difference 0.066
4: I've marked YWRYLDDEDYAAK 99, index is 3833, similarity 0.692, Mass difference 22.045
3: I've marked YVWYRDKNDYAAK 99, index is 3701, similarity 0.692, Mass difference 15.943
1: I've marked RLMHFDWPDYAEK 94, index is 3834, similarity 0.692, Mass difference 32.03
3: I've marked WRMDLHDYSHQDK 99, index is 3969, similarity 0.692, Mass difference 21.964
3: I've marked YLDMHDYSFHRLK 99, index is 4005, similarity 0.692, Mass difference 32.017
1: I've marked RTWRKDLEDYAQK 90, index is 3843, similarity 0.692, Mass difference 16.004
1: I've marked WVDRHDYANYDEK 97, index is 3852, similarity 0.692, Mass difference 31.854
1: I've marked WVDRHDYANYDEK 97, index is 3852, similarity 0.692, Mass difference 15.976
2: I've marked FSEDQHDYSE

4: I've marked YLDMHDYSFHRLK 99, index is 4005, similarity 0.692, Mass difference 32.017
2: I've marked NRDDMHDYSYFRK 86, index is 4083, similarity 0.692, Mass difference 21.941
2: I've marked FRSMYDMPDYSWK 88, index is 4071, similarity 0.846, Mass difference 15.994
2: I've marked MYWHDRHDYARAK 91, index is 4090, similarity 0.692, Mass difference 21.979
3: I've marked RWDLQDYAWYQGK 98, index is 3956, similarity 0.692, Mass difference 16.063
2: I've marked FRKWDTRDYAYPK 92, index is 4024, similarity 0.846, Mass difference 1.001
4: I've marked RWQYDLEDYAQMK 93, index is 4022, similarity 0.692, Mass difference 21.972
1: I've marked FRKWDTRDYAYPK 92, index is 4024, similarity 0.846, Mass difference 1.001
1: I've marked WNMFQDLHDYAYK 98, index is 4026, similarity 0.692, Mass difference 15.933
4: I've marked HLTWQWDREDYAK 99, index is 4036, similarity 0.692, Mass difference 31.983
2: I've marked WNLDKHDYSEKWK 94, index is 4043, similarity 0.692, Mass difference 0.057
2: I've marked WNDMHDYSH

In [10]:
# Remove peptides marked in Step 2 above, making a new dataframe
unique_to_drop = list(np.unique(to_drop))
new_index = list(df.index.values)
for bad_pep in unique_to_drop:
    new_index.remove(bad_pep)
df_intermed = df.iloc[new_index,:]
df_intermed.reset_index(drop=True,inplace=True)
print(f'Step 2 completed, removed {len(peptides)-len(df_intermed.index)} isomers')

Step 2 completed, removed 1568 isomers


In [11]:
# Step 3: Remove sequence isomers by high degree of similarity 
    # Similarity > 0.92, which is seen only for simple swap of 2 amino acids
# THIS IS AN EXTREMELY CONSERVATIVE STEP, sure to remove any POSSIBLE sequence isomers. On clean data, it removes <5% of the data
# RT cannot be used because different gradients were used to collect this data, but is reported

# Extract lists from dataframe for enumeration
peptides = df_intermed['Peptide'].tolist()
OPM = df_intermed['Obs Parent Mass'].tolist()
ALC = df_intermed['ALC (%)'].tolist()
ppm = df_intermed['ppm'].tolist()
RT = df_intermed['ppm'].tolist()

# Make a list of all the peptides to remove, Step 3:
similarity_cutoff = 0.92      # more than this

to_drop = []
for i in range(len(peptides)):
    if len(peptides) > 500 and i % 100 == 0: print(f"Step 3: {i} of {len(peptides)}")
    for k in range(len(peptides)):
        sim = difflib.SequenceMatcher(a=peptides[k], b=peptides[i])
        simz = sim.ratio()
        difference = np.abs(float(OPM[i]) - float(OPM[k]))
        RT_difference = np.abs(float(RT[i]) - float(RT[k]))
        if i != k and simz > similarity_cutoff:
            #print(f"Trouble {peptides[k]} {ALC[k]} {round(float(OPM[k]),3)} and {peptides[i]} {ALC[i]} {round(float(OPM[i]),3)}, similarity {round(float(simz),3)} difference {round(float(difference),3)}")
            if ALC[k] > ALC[i]:
                print(f"1: I've marked {peptides[i]} {ALC[i]}, index is {i}, similarity {round(float(simz),3)}, Mass difference {round(float(difference),3)}, RT difference {round(float(RT_difference),2)}")
                to_drop.append(i)
            elif ALC[k] < ALC[i]:
                print(f"2: I've marked {peptides[k]} {ALC[k]}, index is {k}, similarity {round(float(simz),3)}, Mass difference {round(float(difference),3)}, RT difference {round(float(RT_difference),2)}")
                to_drop.append(k)
            elif ALC[k] == ALC[i]:    
                if abs(float(ppm[k])) < abs(float(ppm[i])):
                    print(f"3: I've marked {peptides[k]} {ALC[k]}, index is {k}, similarity {round(float(simz),3)}, Mass difference {round(float(difference),3)}, RT difference {round(float(RT_difference),2)}")
                    to_drop.append(k)
                elif abs(float(ppm[i])) < abs(float(ppm[k])):
                    print(f"4: I've marked {peptides[i]} {ALC[i]}, index is {i}, similarity {round(float(simz),3)}, Mass difference {round(float(difference),3)}, RT difference {round(float(RT_difference),2)}")
                    to_drop.append(i)
                else:
                    print(f"5: I've marked {peptides[k]} {ALC[k]}, index is {k}, similarity {round(float(simz),3)}, Mass difference {round(float(difference),3)}, RT difference {round(float(RT_difference),2)}")
                    to_drop.append(k)

Step 3: 0 of 2633
4: I've marked GSASGLSFKGSHK 99, index is 2, similarity 0.923, Mass difference 26.048, RT difference 3.2
3: I've marked GSASGLSFKGSHK 99, index is 2, similarity 0.923, Mass difference 26.048, RT difference 3.2
2: I've marked ELASGLSFKGSHK 96, index is 20, similarity 0.923, Mass difference 72.021, RT difference 0.3
1: I've marked ELASGLSFKGSHK 96, index is 20, similarity 0.923, Mass difference 72.021, RT difference 0.3
2: I've marked VNPVMDVGDYAEK 98, index is 175, similarity 0.923, Mass difference 28.035, RT difference 2.6
Step 3: 100 of 2633
1: I've marked VNPVMDVGDYAEK 98, index is 175, similarity 0.923, Mass difference 28.035, RT difference 2.6
4: I've marked EVDLSDYADKVGK 99, index is 185, similarity 0.923, Mass difference 34.019, RT difference 0.9
Step 3: 200 of 2633
Step 3: 300 of 2633
3: I've marked EVDLSDYADKVGK 99, index is 185, similarity 0.923, Mass difference 34.019, RT difference 0.9
Step 3: 400 of 2633
4: I've marked LMGFQDVQDYAAK 99, index is 424, simil

In [12]:
# Remove peptides marked in Step 3 above, making a new dataframe
unique_to_drop = list(np.unique(to_drop))
new_index = list(df_intermed.index.values)
for bad_pep in unique_to_drop:
    new_index.remove(bad_pep)
df_final = df_intermed.iloc[new_index,:]
df_final.reset_index(drop=True,inplace=True)
print(f'Step 3 completed, removed {len(peptides)-len(new_index)} potential isomers')

Step 3 completed, removed 12 potential isomers


In [13]:
# Save dataframe without index
df_final.to_csv(f'{file[:-4]}_clean.csv', index=False)
print(f"Output saved, of {len(ALC_list)} peptides originally present, {len(new_index)} were retained")

Output saved, of 4267 peptides originally present, 2621 were retained
